# Local Government Data
### from https://coronavirus.data.gov.uk/details/download


In [18]:
area = ['Solihull', 'Ashfield', 'Daventry']
url = ['https://api.coronavirus.data.gov.uk/v2/data?areaType=ltla&areaCode=E08000029&metric=newCasesBySpecimenDateAgeDemographics&format=csv',
      'https://api.coronavirus.data.gov.uk/v2/data?areaType=ltla&areaCode=E07000170&metric=newCasesBySpecimenDateAgeDemographics&format=csv',
      'https://api.coronavirus.data.gov.uk/v2/data?areaType=ltla&areaCode=E07000151&metric=newCasesBySpecimenDateAgeDemographics&format=csv']


i = 1
for place in area:
    print(f'{i:2.0f} > {area[i-1]}')
    i+=1

opt = int(input('Choose number: '))

nhsdata = url[opt-1]
location = area[opt-1]



 1 > Solihull
 2 > Ashfield
 3 > Daventry
Choose number: 2


In [19]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## first the permanent link for downloading, based on user choice
df = pd.read_csv(nhsdata)


In [20]:
## This is the output file
html = open(f'{location}_COVID.html', 'w')

## the grouped counts we want to drop, they will cause double counting.
df1 = df.query("age not in ['00_59', '60+', 'unassigned']")

## convert string date to a real date format
df1['date'] = pd.to_datetime(df1['date'])

## calculate the most recent date 
latest = df1.date.max().strftime("%Y-%m-%d")

## Create the first Bar Chart
fig = px.bar(df1,
            x='date',
            y='cases',
            color='age',
            title=f'{location} - New Cases per day - by Age Group (UK Government Data)')

## Now we need to process the data to get x & y for total new cases per day
dfx = df1.date.unique()[::-1]    # note we have to reverse the date list, otherwise graph is wrong direction. 
dfy = df1.groupby('date').cases.sum()
## Create a second chart with scatter and then the trend line, we are going to use the trend line to add to the first graph
fig2 = px.scatter(x=dfx, y=dfy, 
                      color_discrete_sequence=['#000000'],
                      trendline="rolling", trendline_options=dict(window=7))

## add the trend line -- data[1] -- to the first chart.
fig.add_trace(fig2.data[1])

# Add range slider and data selecter buttons as well. 
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=14, label="fortnight", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True), type="date"
    )
)


fig.show()

## write the file out
html.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

html.close()


<ipython-input-20-2c76b2bb7427>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [21]:
html

<_io.TextIOWrapper name='Ashfield_COVID.html' mode='w' encoding='cp1252'>

NameError: name 'plotly' is not defined